In [2]:
import pandas as pd
import statistics
import numpy as np
from sklearn import preprocessing, cluster, metrics, decomposition
import matplotlib.pyplot as plt
import kneed
from scipy.cluster.hierarchy import dendrogram

https://realpython.com/k-means-clustering-python/ <br>
https://towardsdatascience.com/interpretable-k-means-clusters-feature-importances-7e516eeb8d3c

# Data Preparations

Import data from an Excel file.

In [ ]:
#Import the data file
data = pd.read_excel('../LettuceData.xlsx')

In [ ]:
#View column names
data.columns

In [ ]:
#View data
data

Experiments were run multiple times per lettuce type. Average the experiment values per lettuce type. Create a dataframe with the lettuce name as index, the experiment type as column name and the average value for this experiment for a lettuce type as value.

In [ ]:
experiments = set(data['Experiment'].values) #Get all the different experiments
sla = [i for i in list(data.columns) if i not in ['Experiment','Number']] #Get lettuce names
dt = pd.DataFrame(index=sla,columns = experiments) #Format dataframe with lettuce names as row index and experiments as columns

#Iterate over the experiments and lettuces
#Calculate and assign the mean of different runs of experiments per lettuce
for e in experiments:
    for s in sla:
            numbers = set(data[data['Experiment']==e]['Number'].values)
            value = np.nanmean(list(data[data['Experiment']==e][s].values))
            dt.at[s,e] = value

In [ ]:
#View the new dataframe
dt

Data ranges of the experiments varied a lot. Therefore, values for each experiment were scaled to the range [0, 1]. The smallest value was assigned 0, the largest 1 and the other values were scaled proportionally between this minimum and maximum.  

In [ ]:
#Scale the values in each column between 0 and 1
#Replace the original values with the scaled values
x = dt.values
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
dt = pd.DataFrame(x_scaled, index=dt.index, columns = dt.columns)

In [ ]:
#View the dataframe
dt

# K-means clustering

Apply K-means clustering to the dataset, try models with 1 up to 10 clusters

In [ ]:
#Initialize the k means model
kmeans_kwargs = {"init": "random","n_init": 10,"max_iter": 300,"random_state": 42,} 
# A list to hold the SSE values for each k
sse = []
#Create models with 1 up to 10 clusters
for k in range(1, 11):
    kmeans = cluster.KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(dt)
    sse.append(kmeans.inertia_)

In [ ]:
#Plot the squared sum of distances for each model with a different number k, i.e. different number of clusters
plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1,11))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()

In [ ]:
#Sum of squared errors
sse

Determine the best number of clusters for this dataset with the elbow method.

In [ ]:
#Using the elbow method, find the point of inflection
kl = kneed.KneeLocator(range(1, 11), sse, curve="convex", direction="decreasing")
kl.elbow

To verify the choice for this number of clusters, the Silhouette score can also be computed. It quantifies how well data points are clusterd by comparing data points that are similar to each other based on intra-cluster distances and nearest-cluster distances.

In [ ]:
silhouette_coefficients = [] #List to store silhouette coeficients of each model
# Notice you start at 2 clusters for silhouette coefficient
for k in range(2, 11):
    kmeans = cluster.KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(dt)
    score = metrics.silhouette_score(dt, kmeans.labels_)
    silhouette_coefficients.append(score)

In [ ]:
#Plot the silhouette coefficients for each model with a different number k, i.e. different number of clusters
plt.style.use("fivethirtyeight")
plt.plot(range(2, 11), silhouette_coefficients)
plt.xticks(range(2, 11))
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Coefficient")
plt.show()

In [ ]:
silhouette_coefficients

# Create Cluster Model

3 clusters seems to be the best choice based on both the elbow method and the silhouette coefficient analysis.
Therefore, create a k-means clustering model with 3 clusters

In [ ]:
kmeans = cluster.KMeans(n_clusters=3, **kmeans_kwargs)
kms = kmeans.fit(dt)

In [ ]:
#Get the cluster to which each data point belongs and add it to the dataframe
kms.labels_
dt['Cluster'] = kms.labels_

In [ ]:
#Print dataframe with the cluster label in the final column
dt

Visualize the clusters by plotting each type of lettuce on two features/experiments. The 3 different clusters are represented by 3 different colors.

In [ ]:
#Plot clusters for each possible combination of 2 features (2 experiments) and assign a color to each cluster
features = list(dt.columns)[:-1]
for i in range(0,len(features)):
    for j in range(i,len(features)):
        f1 = (features[i])
        f2 = (features[j])
        if f1 != f2:
            fig = plt.figure()
            ax = fig.add_subplot(111)
            scatter = ax.scatter(dt[f1],dt[f2],c=dt['Cluster'],s=50)
            ax.set_title('K-Means Clustering')
            ax.set_xlabel(f1)
            ax.set_ylabel(f2)
            plt.colorbar(scatter)

In [ ]:
#Print the feature names
features = dt.columns.tolist()[0:-1]
print(f"Features: \n{features}")

#Print the centroids' values for the features. Centroids are the middle points in the clusters
centroids = kms.cluster_centers_
print(f"Centroids \n{centroids}")

In [ ]:
#View for each centroid the features in descending order (feature with highest value for that centroid first) 
sorted_centroid_features_idx = centroids.argsort(axis=1)[:,::-1]
print(f"Sorted Feature/Dimension Indexes for Each Centroid in Descending Order: \n{sorted_centroid_features_idx}")

print()

#Print for each centroid the feature name and value in descending order of feature value
sorted_centroid_features_values = np.take_along_axis(centroids, sorted_centroid_features_idx, axis=1)

print('Centroids with features and values in descending order:')

first_features_in_centroid_1 = centroids[0][sorted_centroid_features_idx[0]]
print(list(zip([features[feature] for feature in sorted_centroid_features_idx[0]],first_features_in_centroid_1)))

print()

first_features_in_centroid_2 = centroids[1][sorted_centroid_features_idx[1]]
print(list(zip([features[feature] for feature in sorted_centroid_features_idx[1]],first_features_in_centroid_2)))

print()

first_features_in_centroid_3 = centroids[2][sorted_centroid_features_idx[2]]
print(list(zip([features[feature] for feature in sorted_centroid_features_idx[2]],first_features_in_centroid_3)))

# Hierarchical Clustering

Also visualize the 3 clusters using agglomerative hierarchical clustering. Initially every data point is its own cluster and every time the two clusters that are most similar are merged into 1 cluster.

In [ ]:
#Function to plot the dendogram of the hierarchical clustering
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)
    print(linkage_matrix)
    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix,leaf_rotation=90,labels=['Medium Supermarkt','Butterhead','Lollo Rosso','Red iceberg',
                                                       'Stalk Lettuce','Medium Selfgrown','L. sativa cv. Olof',
                                                       'L. sativa cv Salinas','L. serriola','L. saligna','L. virosa'], **kwargs)

In [ ]:
hclust_dist = []
#for k in range(1, 11):
hclust = cluster.AgglomerativeClustering(n_clusters=3).fit(dt)
print(hclust.n_clusters_)
hclust = cluster.AgglomerativeClustering(n_clusters=3,compute_distances=True).fit(dt)
plot_dendrogram(hclust)

In [ ]:
#Add the hierarchical clustering labels to the dataset
dt_withlabels = dt.copy()
dt_withlabels['Agglomerative clusters'] = hclust.labels_

In [ ]:
#Print the dataframe with the hierarchical cluster labels in the final column
dt_withlabels

# Principal Component Analysis

Reduce the information from all the experiments to two dimensions using Principal Component Analysis (PCA) to be able to plot it in a two-dimensional graph.

In [ ]:
#Reduce the information contained in all features to 2 dimensions
pca = decomposition.PCA(n_components=2,svd_solver='full')
dt_sub = dt.drop(columns=['Cluster'])

In [ ]:
#Apply pca to the average scaled experiment values
X_sub = dt_sub.values
X_pca = pca.fit_transform(X_sub)
print('PCA explained variance ratio {}'.format(pca.explained_variance_ratio_))
print('PCA singular values {}'.format(pca.singular_values_))

In [ ]:
dt_pca = pd.DataFrame(X_pca,columns=['Dimension 1','Dimension 2'])
dt_pca['Cluster'] = dt['Cluster'].values

In [ ]:
dt_pca

In [ ]:
#Plot the 2 dimensions and assign a color to each cluster
fig = plt.figure()
ax = fig.add_subplot(111)
scatter = ax.scatter(dt_pca['Dimension 1'],dt_pca['Dimension 2'],c=dt_pca['Cluster'],s=50)
ax.set_title('PCA')
ax.set_xlabel('Dimension 1')
ax.set_ylabel('Dimension 2')
plt.colorbar(scatter)